In [ ]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import CodeInterpreterTool
from pathlib import Path
from azure.identity import AzureCliCredential
from dotenv import load_dotenv
from pathlib import Path

# ---------------------------------------------------------------------------
# Load Environment Variables
# ---------------------------------------------------------------------------

# Load environment variables from the .env file
env_path = Path().resolve().parent.parent / ".env"
load_dotenv(dotenv_path=env_path)

# Retrieve required environment variables
PROJECT_CONNECTION_STRING = "eastus2.api.azureml.ms;0f737c80-e1bf-4878-99b9-a92a56bf1c9e;basicrg713;florianfollonier7-1050" #os.getenv("Azure_AI_PROJECT_CONNECTION_STRING")

# Ensure that required variables are set
if not PROJECT_CONNECTION_STRING:
    raise ValueError("PROJECT_CONNECTION_STRING is not set in the .env file.")
print("Environment variables loaded successfully.")

project_client = AIProjectClient.from_connection_string(
    credential=AzureCliCredential(), conn_str=PROJECT_CONNECTION_STRING
)

with project_client:
    # Create an instance of the CodeInterpreterTool
    code_interpreter = CodeInterpreterTool()

    # The CodeInterpreterTool needs to be included in creation of the agent
    agent = project_client.agents.create_agent(
        model="gpt-4o",
        name="my-agent",
        instructions="You are helpful agent",
        # tools=code_interpreter.definitions,
        # tool_resources=code_interpreter.resources,
    )
    print(f"Created agent, agent ID: {agent.id}")

    # Create a thread
    thread = project_client.agents.create_thread()
    print(f"Created thread, thread ID: {thread.id}")

    # Create a message
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="Hello, can you count from 1 to 10?",
    )
    print(f"Created message, message ID: {message.id}")

    # Run the agent
    run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        # Check if you got "Rate limit is exceeded.", then you want to get more quota
        print(f"Run failed: {run.last_error}")

    # Get messages from the thread
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

    # Get the last message from the sender
    last_msg = messages.get_last_text_message_by_role("assistant")
    if last_msg:
        print(f"Last Message: {last_msg.text.value}")

    # Generate an image file for the bar chart
    for image_content in messages.image_contents:
        print(f"Image File ID: {image_content.image_file.file_id}")
        file_name = f"{image_content.image_file.file_id}_image_file.png"
        project_client.agents.save_file(file_id=image_content.image_file.file_id, file_name=file_name)
        print(f"Saved image file to: {Path.cwd() / file_name}")

    # Print the file path(s) from the messages
    for file_path_annotation in messages.file_path_annotations:
        print(f"File Paths:")
        print(f"Type: {file_path_annotation.type}")
        print(f"Text: {file_path_annotation.text}")
        print(f"File ID: {file_path_annotation.file_path.file_id}")
        print(f"Start Index: {file_path_annotation.start_index}")
        print(f"End Index: {file_path_annotation.end_index}")
        project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=Path(file_path_annotation.text).name)

    # Delete the agent once done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")